<center>
    <h1> Scientific Programming in Python  </h1>
    <h2> Topic 4: Just in Time Compilation: Numba and NumExpr </h2> 
</center>

_Notebook created by Martín Villanueva - `martin.villanueva@usm.cl` - DI UTFSM - April 2017._

In [3]:
import numba
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

En esta actividad implementaremos una conocida métrica para medir disimilitud entre conjuntos: __La métrica de Hausdorff__. Esta corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 


Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.

1. Implemente la métrica de Hausdorff en Python clásico.
2. Implemente la métrica de Hausdorff usando Numba (Forzando el modo __nopython__ y definiendo explícitamente las _signatures_ de las funciones).
3. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las funciones anteriores sobre estos arreglos.
4. Concluya.

### Python Clásico

In [92]:
def hausdorff_clasico(X, Y):
    xrows = X.shape[0]
    yrows = Y.shape[0]
    dist = np.empty((xrows, yrows), dtype="float64")
    for i in range(xrows):
        for j in range(yrows):
            dist[i][j] = np.linalg.norm(X[i]-Y[j])
    maxxmin = dist.min(axis=0).max()
    maxymin = dist.min(axis=1).max()
    return np.max([maxxmin, maxymin])
    

### Numba

In [126]:
@numba.jit('float64 (float64[:,:], float64[:,:])', nopython=True)
def hausdorff_numba(X, Y):
    xrows = X.shape[0]
    yrows = Y.shape[0]
    dist = np.empty((xrows,yrows), dtype=np.float64)
    xmin = np.empty(xrows, dtype=np.float64)
    ymin = np.empty(yrows, dtype=np.float64)
    for i in range(xrows):
        for j in range(yrows):
            dist[i][j] = np.linalg.norm(X[i]-Y[j])
        xmin[i] = dist[i,:].min()
    for j in range(yrows):
        ymin[j] = dist[:,j].min()
    maxx = xmin.max()
    maxy = ymin.max()
    if maxx > maxy:
        return maxx
    return maxy

### Pruebas

In [130]:
Xs = []
Ys = []
for i in range(1,11):
    X = np.random.rand(i*100*3).reshape(i*100,3)*100
    Y = np.random.rand(i*120*3).reshape(i*120,3)*100
    Xs.append(X)
    Ys.append(Y)

In [132]:
pruebas = dict()
for i in range(10):
    print("Experimento",i+1)
    print("X:({0},{1}),Y:({2},{1})".format(100*(i+1),3,120*(i+1)))
    Xi = Xs[i]
    Yi = Ys[i]
    print("Clasico:")
    cls = %timeit -o hausdorff_clasico(Xi, Yi)
    print("Numba:")
    num = %timeit -o hausdorff_numba(Xi,Yi)
    print("\n")
    pruebas[i] = (cls, num)

Experimento 1
X:(100,3),Y:(120,3)
Clasico:
1 loop, best of 3: 328 ms per loop
Numba:
100 loops, best of 3: 11.2 ms per loop


Experimento 2
X:(200,3),Y:(240,3)
Clasico:
1 loop, best of 3: 1.31 s per loop
Numba:
10 loops, best of 3: 45 ms per loop


Experimento 3
X:(300,3),Y:(360,3)
Clasico:
1 loop, best of 3: 2.93 s per loop
Numba:
10 loops, best of 3: 101 ms per loop


Experimento 4
X:(400,3),Y:(480,3)
Clasico:
1 loop, best of 3: 5.16 s per loop
Numba:
10 loops, best of 3: 179 ms per loop


Experimento 5
X:(500,3),Y:(600,3)
Clasico:
1 loop, best of 3: 8.13 s per loop
Numba:
1 loop, best of 3: 285 ms per loop


Experimento 6
X:(600,3),Y:(720,3)
Clasico:
1 loop, best of 3: 11.7 s per loop
Numba:
1 loop, best of 3: 405 ms per loop


Experimento 7
X:(700,3),Y:(840,3)
Clasico:
1 loop, best of 3: 16.1 s per loop
Numba:
1 loop, best of 3: 568 ms per loop


Experimento 8
X:(800,3),Y:(960,3)
Clasico:
1 loop, best of 3: 21 s per loop
Numba:
1 loop, best of 3: 725 ms per loop


Experimento 9
X:(

#### Conclusion
Efectivamente, el utilizar numba aunque puede costar el uso de funciones de numpy, reduce bastante el tiempo de ejecución de una función.